In [1]:
import numpy as np
import os
import cv2
import tensorflow as tf
from sklearn.model_selection import KFold

In [2]:
# Establecer los valores de OSNR para cada espaciamiento de canal
osnr = {
    "single_ch": [14.3, 15.3, 16.3, 17.8, 19.3, 21.3, 23.3, 26.3, 28.3, 31.3, 36.3],
    "18GHz_spacing": [18, 19, 20, 23, 25, 27, 30, 32, 35, 40],
    "17.6GHz_spacing": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "17GHz_spacing": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "16.5GHz_spacing": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "16GHz_spacing": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "15.5GHz_spacing": [20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "15GHz_spacing": [23, 25, 27, 30, 32, 35, 40],
}

In [3]:
# Arreglos para almacenar las imágenes y etiquetas
images = []
labels = []

# Cargar las imágenes y etiquetas
for spacing in osnr.keys():
    directory = f"constellations/{spacing}/"
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            path = os.path.join(directory, filename)
            image = cv2.imread(path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.GaussianBlur(image, (3, 3), 0)
            image = image / 255.0
            images.append(image)
            if spacing == "single_ch":
                labels.append(0)
            elif spacing == "18GHz_spacing":
                labels.append(0)
            elif spacing == "17.6GHz_spacing":
                labels.append(1)
            elif spacing == "17GHz_spacing":
                labels.append(1)
            elif spacing == "16.5GHz_spacing":
                labels.append(1)
            elif spacing == "16GHz_spacing":
                labels.append(1)
            elif spacing == "15.5GHz_spacing":
                labels.append(1)
            elif spacing == "15GHz_spacing":
                labels.append(1)

images = np.array(images).astype("float32")
labels = np.array(labels).astype("uint8")
input_shape = (images[0].shape[0], images[0].shape[1], 3)
num_classes = len(np.unique(labels))

In [4]:
print("Input shape:", input_shape)
print("Number of classes:", num_classes)

Input shape: (64, 64, 3)
Number of classes: 2


In [5]:
def create_neuronal_network(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=input_shape),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation="selu"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [6]:
# Hacer validacion cruzada
kf = KFold(n_splits=10, shuffle=True)

# Arreglos para almacenar los resultados
accuracies = []
losses = []
models = []
histories = []

# Entrenar el modelo
for train_index, test_index in kf.split(images):
    x_train, x_test = images[train_index], images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Crear el modelo
    model = create_neuronal_network(input_shape, num_classes)

    history = model.fit(x_train, y_train, epochs=300, validation_data=(x_test, y_test))

    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    accuracies.append(accuracy)
    losses.append(loss)
    models.append(model)
    histories.append(history)




Epoch 1/300


21/21 [==============================] - 2s 40ms/step - loss: 0.6077 - accuracy: 0.7332 - val_loss: 0.5212 - val_accuracy: 0.7808
Epoch 2/300
21/21 [==============================] - 1s 27ms/step - loss: 0.5546 - accuracy: 0.7363 - val_loss: 0.5342 - val_accuracy: 0.7945
Epoch 3/300
21/21 [==============================] - 1s 27ms/step - loss: 0.5399 - accuracy: 0.7546 - val_loss: 0.5267 - val_accuracy: 0.8082
Epoch 4/300
21/21 [==============================] - 1s 28ms/step - loss: 0.5315 - accuracy: 0.7713 - val_loss: 0.5320 - val_accuracy: 0.7808
Epoch 5/300
21/21 [==============================] - 1s 28ms/step - loss: 0.5291 - accuracy: 0.7683 - val_loss: 0.5219 - val_accuracy: 0.7945
Epoch 6/300
21/21 [==============================] - 1s 28ms/step - loss: 0.5302 - accuracy: 0.7774 - val_loss: 0.5172 - val_accuracy: 0.8082
Epoch 7/300
21/21 [==============================] - 1s 29ms/step - loss: 0.5336 - accuracy: 0.7698 - val_loss: 0.5124 - val_accuracy: 0.7945
E

In [7]:
# Evaluar los modelos guardados en la validacion cruzada y seleccionar el mejor
best_model_index = np.argmax(accuracies)
best_model = models[best_model_index]
best_model_accuracy = accuracies[best_model_index]
best_model_loss = losses[best_model_index]
print("Best model accuracy:", best_model_accuracy * 100, "%")
print("Best model loss:", best_model_loss)

Best model accuracy: 93.1506872177124 %
Best model loss: 0.5195790529251099


In [8]:
dir = "modelos/entrenamiento_clases/exp_2_kfold10_300epocas/tomado_17.6GHz/"

# Guardar accuracy y loss del mejor modelo
a = [best_model_accuracy, best_model_loss]
np.save(f"{dir}{num_classes}classes_acc_loss_traslape.npy", a)

# Guardar el mejor modelo
best_model.save(f"{dir}{num_classes}classes_best_model_traslape.h5")

# Guardar la mejor historia de entrenamiento del modelo
best_model_history = histories[best_model_index].history
np.save(f"{dir}{num_classes}classes_history_traslape.npy", best_model_history)

c:\Users\martinez\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
